# Simulação de um Sistema Solar com um Vulcano

## Criando arquivos de entrada para simulação

In [35]:
# Import modules
import os
import sys
import yaml
import shutil
from glob import glob
import pandas as pd

## Definindo funções

In [36]:
def read_config_file(config_file):
    """
    Read config file and go to home directory
    """
    home_dir = os.path.dirname(os.path.abspath(config_file))
    os.chdir(home_dir)

    # Read yaml file
    with open(config_file, 'r') as stream:
        try:
            config = yaml.load(stream)
        except yaml.YAMLError as exc:
            print(exc)
    
    return config

In [37]:
def write_pl(pl):
    """
    Write input files
    """
    for i, j in enumerate(glob("*")):
        os.chdir(j)
        with open("pl.in", "a") as f:
            f.write(str(pl.ix[i]["mass_grav"]) + "\n")
            f.write(str(pl.ix[i]["x"]) + " " + str(pl.ix[i]["y"]) + " " + str(pl.ix[i]["z"]) + "\n")
            f.write(str(pl.ix[i]["vx"]) + " " + str(pl.ix[i]["vy"]) + " " + str(pl.ix[i]["vz"]) + "\n")
        os.chdir("../")

## Leitura do arquivo de configuração e definição de variáveis

In [38]:
config_file = "config.yaml"
home_dir = os.path.dirname(os.path.abspath(config_file))
os.chdir(home_dir)

config = read_config_file("config.yaml")
simulation_name = config["simulation_name"]
planets_name = config['planets_names']
vulcan_variants_suffix = config["vulcans_semi_axis"]
vulcan_variants = len(config["vulcans_semi_axis"])
vulcan_clones = config["vulcans_clones"]
subroutine_swift = config["subroutine_swift"]

## Abrindo os dataframes 

### Vulcan

In [39]:
### Open planet's data
# vulcan
vulcan = pd.read_csv("vulcan.csv")

### Planetas

In [40]:
# Other planets
for i in planets_name:
    exec("{0} = pd.read_csv('{0}' + '.csv')".format(i))

## Criando os dados para inserir nos arquivos de configuração

### Sol

In [41]:
# ## Sun's data
sun_data = """10
2.959139768995959E-04
0.0 0.0 0.0
0.0 0.0 0.0
"""
os.chdir(simulation_name)

for i, j in enumerate(glob("*")):
    os.chdir(j)
    with open("pl.in", "a") as f:
        f.write(sun_data)
    os.chdir("../")

os.chdir(home_dir)

### Vulcan

In [42]:
# vulcan
os.chdir(simulation_name)

write_pl(vulcan)

os.chdir(home_dir)

### Outros planetas

In [43]:
# Others planets
os.chdir(simulation_name)
# write input files

for pl in planets_name:
    exec("write_pl({0})".format(pl))

os.chdir(home_dir)

### Partícula de testes

In [44]:
# ## Test particle
# There is no test particle
tp = """
0
"""

os.chdir(simulation_name)

for i, j in enumerate(glob("*")):
    os.chdir(j)
    with open("tp.in", "a") as f:
        f.write(tp)
    os.chdir("../")

os.chdir(home_dir)

### Parâmetros da simulação
param.in

In [45]:
# ## Parameters of the simulation
# 
# Write param.in
initial_time= config["initial_time"]
time_stop= config["time_stop"]
timestep= config["timestep"]
dtout= config["dtout"]
dtdump= config["dtdump"]
L1= config["L1"]
L2= config["L2"]
L3= config["L3"]
L4= config["L4"]
L5= config["L5"]
L6= config["L6"]
rmin= config["rmin"]
rmax= config["rmax"]
rmaxu= config["rmaxu"]
qmin= config["qmin"]
lclose= config["lclose"]
binary_outputfile= config["binary_outputfile"]
status_flag_for_open_statements= config["status_flag_for_open_statements"]

string = """{0} {1} {2}
{3} {4}
{5} {6} {7} {8} {9} {10}
{11} {12} {13} {14} {15}
{16}
{17}""".format(initial_time, time_stop, timestep, dtout, dtdump, \
                        L1, L2, L3, L4, L5, L6, rmin, rmax, rmaxu, qmin, \
                        lclose, binary_outputfile, status_flag_for_open_statements)

os.chdir(simulation_name)

for i, j in enumerate(glob("*")):
    os.chdir(j)
    with open("param.in", "a") as f:
        f.write(string)
    os.chdir("../")

os.chdir(home_dir)

### Argumento para a subrotina do SWIFT
swift_mvs.in

In [46]:
# ## arguments for Swift routine 
os.chdir(simulation_name)

for i, j in enumerate(glob("*")):
    os.chdir(j)
    with open(subroutine_swift, "a") as f:
        f.write("param.in\n")
        f.write("pl.in\n")
        f.write("tp.in")
    os.chdir("../")

os.chdir(home_dir)

## Verificando dados

### Dados no diretório de simulação

In [47]:
glob(simulation_name + "/" + simulation_name + "-A-001/*")

['vulcan_mvs/vulcan_mvs-A-001/pl.in',
 'vulcan_mvs/vulcan_mvs-A-001/tp.in',
 'vulcan_mvs/vulcan_mvs-A-001/param.in',
 'vulcan_mvs/vulcan_mvs-A-001/swift_mvs.in']

### pl.in

In [48]:
with open(simulation_name + "/" + simulation_name + "-A-001/pl.in") as f:
    print(f.read())

10
2.959139768995959E-04
0.0 0.0 0.0
0.0 0.0 0.0
8.88753856795e-10
0.0453384309365 -0.0211649495845 2.24111770861e-06
0.031894690683 0.0699477087913 -4.5207854192e-07
8.88753856795e-10
-0.0356409954462 0.386108939237 -1.14420156276e-05
-0.0274911978704 -0.00247137252562 -2.98458747538e-06
7.24345628644e-10
-0.0528082359638 -0.721589770597 6.44037429562e-05
0.020169394448 -0.00144321508841 -9.93038746929e-07
8.88753856795e-10
-0.932928978827 -0.359864448381 -2.60272221997e-05
0.00617823701402 -0.0160557168372 -1.80434321219e-07
9.5494572852e-11
0.980528403182 -1.16151497678 0.00016637970286
0.0106875677458 0.00899506797889 1.8790001203e-06
2.82474705462e-07
-5.01406544849 -2.12898103637 0.120976893005
0.00286116217368 -0.00658796936244 -3.66380492075e-05
8.4576187896e-08
-1.01264638527 -9.99941754135 0.21416400292
0.00524432911633 -0.000579946093552 -0.000198642119843
1.29188735613e-08
18.076310869 8.38763784834 -0.203029803882
-0.00168426517818 0.00338438651774 3.43914855749e-05
1.5240

### tp.in

In [49]:
with open(simulation_name + "/" + simulation_name + "-A-001/tp.in") as f:
    print(f.read())


0



### param.in

In [50]:
with open(simulation_name + "/" + simulation_name + "-A-001/param.in") as f:
    print(f.read())

0.0e0 3.65249999999e8 0.2
3.6525e5 3.6525e5
F T F T T F
-1.0 1000.0 -1.0 -1.0 F
bin.dat
unknown


### swift_mvs.in

In [51]:
with open(simulation_name + "/" + simulation_name + "-A-001/swift_mvs.in") as f:
    print(f.read())

param.in
pl.in
tp.in
